In [1]:
import pandas as pd
import numpy as np

In [2]:
def preprocess(df):
    for i in range(1, 301):
        customer_data = df[df['Customer'] == i]
        customer_data = customer_data.drop(columns=['Generator Capacity', 'Postcode'])

        try:
            customer_data = pd.melt(customer_data, id_vars=['Customer', 'date', 'Consumption Category', 'Row Quality'], var_name='Time')
        except:
            customer_data = pd.melt(customer_data, id_vars=['Customer', 'date', 'Consumption Category'], var_name='Time')
        #customer_data.to_csv('customer_data.csv', index=False)

        customer_data['date'] = customer_data['date'].astype(str)
        zero_padded_date = []
        for date in customer_data['date']:
            if len(date) < 9:
                new_date = '0' + date
                zero_padded_date.append(new_date)
            else:
                zero_padded_date.append(date)

        customer_data['date'] = zero_padded_date

        customer_data['Time'] = customer_data['Time'].astype(str)

        customer_data['datetime'] = customer_data['date'] + ' ' + customer_data['Time']
        try:
            customer_data['datetime'] = pd.to_datetime(customer_data['datetime'], format='%d-%b-%y %H:%M')
        except:
            customer_data['datetime'] = pd.to_datetime(customer_data['datetime'], format='%d/%m/%Y %H:%M')
        customer_data = customer_data.sort_values('datetime')
        #customer_data.to_csv('customer_data_new.csv', index=False)
        year = customer_data['datetime'].iloc[0].year
        year_interval = str(year) + '-' + str(year + 1)
        file_name = 'NL_' + year_interval + '_' + str(i) + '.csv'

        # Separate consumption categories
        gc_data = customer_data[customer_data['Consumption Category'] == 'GC']
        cl_data = customer_data[customer_data['Consumption Category'] == 'CL']
        gg_data = customer_data[customer_data['Consumption Category'] == 'GG']

        gc_data = gc_data.rename(columns = {'value': 'GC'})
        cl_data = cl_data.rename(columns = {'value': 'CL'})
        gg_data = gg_data.rename(columns = {'value': 'GG'})

        gc_data = gc_data.drop(columns=['Consumption Category'])
        cl_data = cl_data.drop(columns=['Consumption Category'])
        gg_data = gg_data.drop(columns=['Consumption Category'])

        gc_data.to_csv('data_gc.csv', index=False)
        cl_data.to_csv('data_cl.csv', index=False)
        gg_data.to_csv('data_gg.csv', index=False)

        if cl_data.empty == False:
            nl_data = pd.merge(gc_data, gg_data, on=['Customer', 'date', 'Time', 'datetime'])
            nl_data = pd.merge(nl_data, cl_data, on=['Customer', 'date', 'Time', 'datetime'])
            nl_data['NL'] = nl_data['GC'] + nl_data['CL'] - nl_data['GG']
            nl_data.to_csv('processed_data/' + year_interval + '/' + file_name, index=False)
        else:
            nl_data = pd.merge(gc_data, gg_data, on=['Customer', 'date', 'Time', 'datetime'])
            nl_data['NL'] = nl_data['GC'] - nl_data['GG']
            nl_data.to_csv('processed_data/' + year_interval + '/' + file_name, index=False)

In [3]:
missing_data = pd.read_csv('raw_data/missing_data_test.csv', skiprows=1)
missing_values = missing_data.isnull()
print(missing_values.value_counts())

Customer  Generator Capacity  Postcode  Consumption Category  date   00:30  01:00  01:30  02:00  02:30  03:00  03:30  04:00  04:30  05:00  05:30  06:00  06:30  07:00  07:30  08:00  08:30  09:00  09:30  10:00  10:30  11:00  11:30  12:00  12:30  13:00  13:30  14:00  14:30  15:00  15:30  16:00  16:30  17:00  17:30  18:00  18:30  19:00  19:30  20:00  20:30  21:00  21:30  22:00  22:30  23:00  23:30  00:00
False     False               False     False                 False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False    269730
                                                                                                                                                                                      

In [4]:
y1_data = pd.read_csv('raw_data/2010-2011 Solar home electricity data.csv', skiprows=1)
print(y1_data.head())

   Customer  Generator Capacity  Postcode Consumption Category      date  \
0         1                3.78      2076                   GC  1-Jul-10   
1         1                3.78      2076                   CL  1-Jul-10   
2         1                3.78      2076                   GG  1-Jul-10   
3         1                3.78      2076                   GC  2-Jul-10   
4         1                3.78      2076                   CL  2-Jul-10   

    0:30   1:00   1:30   2:00   2:30  ...  19:30  20:00  20:30  21:00  21:30  \
0  0.303  0.471  0.083  0.121  0.361  ...  0.495   0.54  0.406  0.543  0.495   
1  1.250  1.244  1.256  0.744  0.019  ...  0.000   0.00  0.000  0.000  0.000   
2  0.000  0.000  0.000  0.000  0.000  ...  0.000   0.00  0.000  0.000  0.000   
3  0.116  0.346  0.122  0.079  0.120  ...  0.262   1.10  1.012  0.817  0.526   
4  1.238  1.238  1.256  1.250  0.169  ...  0.000   0.00  0.000  0.000  0.000   

   22:00  22:30  23:00  23:30   0:00  
0  0.216  0.378  0.128 

In [5]:
y2_data = pd.read_csv('raw_data/2011-2012 Solar home electricity data v2.csv', skiprows=1)
y3_data = pd.read_csv('raw_data/2012-2013 Solar home electricity data v2.csv', skiprows=1)

In [6]:
missing_values = y1_data.isnull()
print(missing_values.value_counts())

Customer  Generator Capacity  Postcode  Consumption Category  date   0:30   1:00   1:30   2:00   2:30   3:00   3:30   4:00   4:30   5:00   5:30   6:00   6:30   7:00   7:30   8:00   8:30   9:00   9:30   10:00  10:30  11:00  11:30  12:00  12:30  13:00  13:30  14:00  14:30  15:00  15:30  16:00  16:30  17:00  17:30  18:00  18:30  19:00  19:30  20:00  20:30  21:00  21:30  22:00  22:30  23:00  23:30  0:00 
False     False               False     False                 False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False    269735
Name: count, dtype: int64


In [7]:
missing_values = y2_data.isnull()
print(missing_values.value_counts())

Customer  Generator Capacity  Postcode  Consumption Category  date   0:30   1:00   1:30   2:00   2:30   3:00   3:30   4:00   4:30   5:00   5:30   6:00   6:30   7:00   7:30   8:00   8:30   9:00   9:30   10:00  10:30  11:00  11:30  12:00  12:30  13:00  13:30  14:00  14:30  15:00  15:30  16:00  16:30  17:00  17:30  18:00  18:30  19:00  19:30  20:00  20:30  21:00  21:30  22:00  22:30  23:00  23:30  0:00   Row Quality
False     False               False     False                 False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  True           270304
Name: count, dtype: int64


In [8]:
missing_values = y3_data.isnull()
print(missing_values.value_counts())

Customer  Generator Capacity  Postcode  Consumption Category  date   0:30   1:00   1:30   2:00   2:30   3:00   3:30   4:00   4:30   5:00   5:30   6:00   6:30   7:00   7:30   8:00   8:30   9:00   9:30   10:00  10:30  11:00  11:30  12:00  12:30  13:00  13:30  14:00  14:30  15:00  15:30  16:00  16:30  17:00  17:30  18:00  18:30  19:00  19:30  20:00  20:30  21:00  21:30  22:00  22:30  23:00  23:30  0:00   Row Quality
False     False               False     False                 False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  False  True           268557
Name: count, dtype: int64


In [9]:
preprocess(y1_data)

In [10]:
preprocess(y2_data)

In [11]:
preprocess(y3_data)